# Import libraries

In [1]:
import pandas as pd
import numpy as np
import os
import config
from joblib import dump, load
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import balanced_accuracy_score, accuracy_score
from GMHI import GMHI

# Load data

In [2]:
X = pd.read_csv(os.path.join(config.CLEAN_DIR, "taxonomic_features.csv"), index_col=[0, 1])
y = pd.read_csv(os.path.join(config.CLEAN_DIR, "metadata.csv"), index_col=[0, 1]).iloc[:, [0]]
display(X.head())
display(y.head())

k__Archaea  k__Archaea|p__Euryarchaeota  \
Study_ID Sample Accession                                            
GMHI-10  SAMN03283239        0.000000                     0.000000   
         SAMN03283266        0.000000                     0.000000   
         SAMN03283281        0.009764                     0.009764   
         SAMN03283294        0.000000                     0.000000   
         SAMN03283288        0.011865                     0.011865   

                           k__Archaea|p__Euryarchaeota|c__Methanobacteria  \
Study_ID Sample Accession                                                   
GMHI-10  SAMN03283239                                            0.000000   
         SAMN03283266                                            0.000000   
         SAMN03283281                                            0.009764   
         SAMN03283294                                            0.000000   
         SAMN03283288                                            0.011865   

                           k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales  \
Study_ID Sample Accession                                                                         
GMHI-10  SAMN03283239                                               0.000000                      
         SAMN03283266                                               0.000000                      
         SAMN03283281                                               0.009764                      
         SAMN03283294                                               0.000000                      
         SAMN03283288                                               0.011865                      

                           k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae  \
Study_ID Sample Accession                                                                                                
GMHI-10  SAMN03283239                                               0.000000                                             
         SAMN03283266                                               0.000000                                             
         SAMN03283281                                               0.009764                                             
         SAMN03283294                                               0.000000                                             
         SAMN03283288                                               0.011865                                             

                           k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae|g__Methanobrevibacter  \
Study_ID Sample Accession                                                                                                                      
GMHI-10  SAMN03283239                                               0.000000                                                                   
         SAMN03283266                                               0.000000                                                                   
         SAMN03283281                                               0.009764                                                                   
         SAMN03283294                                               0.000000                                                                   
         SAMN03283288                                               0.011865                                                                   

                           k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae|g__Methanobrevibacter|s__Methanobrevibacter_smithii  \
Study_ID Sample Accession                                                                                                                                                    
GMHI-10  SAMN03283239                                               0.000000              

PHENOTYPE:Healthy_Nonhealthy
Study_ID Sample Accession                              
GMHI-10  SAMN03283239                              True
         SAMN03283266                              True
         SAMN03283281                              True
         SAMN03283294                              True
         SAMN03283288                              True

# Train Models!

## Train GMHI model on entire dataset

In [3]:
# use same parameters as original paper
gmhi = GMHI(use_shannon=True, theta_f=1.4, theta_d=0.1)

# use only non-viral species
X_GMHI = X.copy()
X_GMHI = X_GMHI[list(filter(lambda x : "s__" in x and "virus" not in x, X_GMHI.columns))]

gmhi.fit(X_GMHI, y)
GMHI_scores = pd.DataFrame(gmhi.decision_function(X_GMHI), index=y.index, columns=["GMHI"])
display(GMHI_scores)
print("GMHI balanced_accuracy on training set:", balanced_accuracy_score(y, GMHI_scores > 0))
idx = (abs(GMHI_scores) >= 0.1).values
print("GMHI balanced_accuracy on training set for |GMHI| >= 0.1:", balanced_accuracy_score(y[idx], GMHI_scores[idx] > 0))
idx = (abs(GMHI_scores) >= 1).values
print("GMHI balanced_accuracy on training set for |GMHI| >= 1:", balanced_accuracy_score(y[idx], GMHI_scores[idx] > 0))

GMHI
Study_ID  Sample Accession          
GMHI-10   SAMN03283239      3.927578
          SAMN03283266      1.797797
          SAMN03283281      3.255258
          SAMN03283294      1.440170
          SAMN03283288      0.877647
...                              ...
Yang_2020 SRR6456373       -4.446721
          SRR6456374       -2.188340
          SRR6456375        0.373152
          SRR6456376        1.185920
          SRR6456377       -0.217462

[8204 rows x 1 columns]

GMHI balanced_accuracy on training set: 0.7072097955955672
GMHI balanced_accuracy on training set for |GMHI| >= 0.1: 0.7117669032086635
GMHI balanced_accuracy on training set for |GMHI| >= 1: 0.7660623915227917


## Train GMHI2 model on entire dataset

In [4]:
gmhi2 = LogisticRegression(random_state=42, penalty="l1", solver="liblinear", C=config.REGULARIZATION, class_weight="balanced")

X_GMHI2 = X > config.PRESENCE_CUTOFF
gmhi2.fit(X_GMHI2, y.values.flatten())
GMHI2_scores = pd.DataFrame(gmhi2.decision_function(X_GMHI2), index=y.index, columns=["GMHI2"])
display(GMHI2_scores)
print("GMHI2 balanced_accuracy on training set:", balanced_accuracy_score(y, GMHI2_scores > 0))
idx = (abs(GMHI2_scores) >= 0.1).values
print("GMHI2 balanced_accuracy on training set |GMHI2| >= 0.1:", balanced_accuracy_score(y[idx], GMHI2_scores[idx] > 0))
idx = (abs(GMHI2_scores) >= 1).values
print("GMHI2 balanced_accuracy on training set |GMHI2| >= 1:", balanced_accuracy_score(y[idx], GMHI2_scores[idx] > 0))


GMHI2
Study_ID  Sample Accession          
GMHI-10   SAMN03283239      1.991018
          SAMN03283266      0.223238
          SAMN03283281      1.053933
          SAMN03283294      1.790399
          SAMN03283288      1.684938
...                              ...
Yang_2020 SRR6456373       -1.023139
          SRR6456374       -0.839538
          SRR6456375       -0.016746
          SRR6456376        0.922137
          SRR6456377       -0.124034

[8204 rows x 1 columns]

GMHI2 balanced_accuracy on training set: 0.7941890692321047
GMHI2 balanced_accuracy on training set |GMHI2| >= 0.1: 0.808233876927126
GMHI2 balanced_accuracy on training set |GMHI2| >= 1: 0.908697248259416


In [5]:
# confusion matrix

print(((GMHI2_scores > 0).values & ~y.values).sum())
print(((GMHI2_scores < 0).values & ~y.values).sum())
print(((GMHI2_scores > 0).values & y.values).sum())
print(((GMHI2_scores < 0).values & y.values).sum())

539
2118
4389
1158


# Export stuff

## Export GMHI scores and serialize model

In [6]:
GMHI_scores.to_csv(os.path.join(config.LOG_DIR, "GMHI_scores_training_set.csv"))
dump(gmhi, os.path.join(config.MODEL_DIR, 'gmhi_model.joblib'))

['../output/models/gmhi_model.joblib']

## Export GMHI2 scores and serialize model

In [7]:
GMHI2_scores.to_csv(os.path.join(config.LOG_DIR, "GMHI2_scores_training_set.csv"))
dump(gmhi2, os.path.join(config.MODEL_DIR, 'gmhi2_model.joblib'))

['../output/models/gmhi2_model.joblib']